In [135]:
import numpy as np

### Helper for coefficients

In [136]:
def coeff_order(monomers, degree, clustersets):
    # monomers      : Number of components in the system
    # degree        : Array with length monomers containing the highest degrees of the components
    # clustersets   : The coefficients are assigned corresponding to the composition of the clusters

    # n_comp : number of coefficients for one polynomial
    n_comp = 1
    for i in range(1, len(degree)+1):
        n_comp = n_comp * degree[i-1]

    # The coeff array has the length of (n_comp * number of components in the system) since 
    # number of components in the system equations are needed to solve the problem
    coeffs = np.zeros(n_comp*monomers)

    # For each equation the first coeff is set to -1.0
    for i in range(0, len(degree)):
        coeffs[i*n_comp] = -1.0

    print(coeffs)

    # Loop over all possible combinations
    for i in range(len(clustersets)):
        index = clustersets[i][0]

        for j in range(1, monomers):
            index += clustersets[i][j] * np.product(degree[:j])

        for k in range(1, monomers):
            coeffs[index + (i-1)*n_comp] = 1

In [137]:
monomers = 2
degree = (4,4)
clusterset = [[1,0], [0,1], [2,0]]

coeff_order(monomers, degree, clusterset)

[-1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
